In [15]:
from utils.handle_data import get_integer_columns_and_dtype, get_cards_metadata, process_and_save_in_batches
import pandas as pd
import numpy as np
import random

In [16]:
d_type = get_integer_columns_and_dtype('Handled_data/draft_data_best_players.csv')
raw_data = pd.read_csv('Handled_data/draft_data_best_players.csv', dtype=d_type)

In [17]:
#groupby draft_id
draft_id_group = raw_data.groupby('draft_id')

In [18]:
#remove all group with less than 42 rows
draft_id_group = draft_id_group.filter(lambda x: len(x) == 42)

In [19]:
draft_id_group

,expansion,event_type,draft_id,draft_time,rank,event_match_wins,event_match_losses,pack_number,pick_number,pick,...,pool_Wose Pathfinder,pool_You Cannot Pass!,pool_Éomer of the Riddermark,"pool_Éomer, Marshal of Rohan","pool_Éowyn, Fearless Knight","pool_Éowyn, Lady of Rohan",pack_card_Dawn of a New Age,pool_Dawn of a New Age,user_n_games_bucket,user_game_win_rate_bucket
0,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,0,Ranger's Firebrand,...,0,0,0,0,0,0,0,0,10,0.60
1,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,1,Gollum's Bite,...,0,0,0,0,0,0,0,0,10,0.60
2,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,2,Erebor Flamesmith,...,0,0,0,0,0,0,0,0,10,0.60
3,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,3,Erebor Flamesmith,...,0,0,0,0,0,0,0,0,10,0.60
4,LTR,PremierDraft,c781c9ecd42b493781f95d69b9737e51,2023-06-15 19:43:44,platinum,7,2,0,4,Hithlain Knots,...,0,0,0,0,0,0,0,0,10,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773956,LTR,PremierDraft,e430b67f33a9487fb7b291453a5de350,2023-07-04 08:45:00,platinum,4,3,2,9,East-Mark Cavalier,...,1,1,0,0,0,0,0,0,100,0.58
1773957,LTR,PremierDraft,e430b67f33a9487fb7b291453a5de350,2023-07-04 08:45:00,platinum,4,3,2,10,Hobbit's Sting,...,1,1,0,0,0,0,0,0,100,0.58
1773958,LTR,PremierDraft,e430b67f33a9487fb7b291453a5de350,2023-07-04 08:45:00,platinum,4,3,2,11,Soldier of the Grey Host,...,1,1,0,0,0,0,0,0,100,0.58
1773959,LTR,PremierDraft,e430b67f33a9487fb7b291453a5de350,2023-07-04 08:45:00,platinum,4,3,2,12,Soldier of the Grey Host,...,1,1,0,0,0,0,0,0,100,0.58


In [20]:
# Seed for reproducibility
np.random.seed(42)

# Get a list of unique group identifiers (i.e., unique 'draft_id's)
unique_ids = draft_id_group['draft_id'].unique()

# Sample 20% of the unique identifiers for the test set
test_ids = np.random.choice(unique_ids, size=int(len(unique_ids) * 0.20), replace=False)

# Create a new list for the remaining ids (those not in test set)
remaining_ids = [id for id in unique_ids if id not in test_ids]

# Sample 25% of the remaining identifiers for the validation set (0.25 * 0.8 = 0.2)
val_ids = np.random.choice(remaining_ids, size=int(len(remaining_ids) * 0.25), replace=False)

# The rest (60% of total) goes to the train set
train_ids = [id for id in remaining_ids if id not in val_ids]

In [21]:
# Create the sets based on the sampled identifiers
test_set = draft_id_group[draft_id_group['draft_id'].isin(test_ids)]
val_set = draft_id_group[draft_id_group['draft_id'].isin(val_ids)]
train_set = draft_id_group[draft_id_group['draft_id'].isin(train_ids)]


#save data
test_set.to_csv('dataset/test_set.csv', index=False)
val_set.to_csv('dataset/val_set.csv', index=False)
train_set.to_csv('dataset/train_set.csv', index=False)

### Add metadata

In [22]:
#Load all cards data
meta_data_card = pd.read_csv('Handled_data/ltr_cards_data.csv')

In [23]:
meta_data_card.head(5)

,name,rarity,color_identity,cmc,colors
0,"Andúril, Flame of the West",mythic,T,3.0,T
1,"Aragorn and Arwen, Wed",mythic,M,6.0,M
2,"Aragorn, Company Leader",rare,M,3.0,M
3,"Aragorn, the Uniter",mythic,M,4.0,M
4,"Arwen, Mortal Queen",mythic,M,3.0,M


In [24]:
meta_data_card = get_cards_metadata('Handled_data/ltr_cards_data.csv')

In [25]:
meta_data_card.head(5)

,name,cmc,id,rarity_common,rarity_mythic,rarity_rare,rarity_uncommon,colors_B,colors_G,colors_M,colors_R,colors_T,colors_U,colors_W
0,"Andúril, Flame of the West",0.002318,0,0,1,0,0,0,0,0,0,1,0,0
1,"Aragorn and Arwen, Wed",1.977281,1,0,1,0,0,0,0,1,0,0,0,0
2,"Aragorn, Company Leader",0.002318,2,0,0,1,0,0,0,1,0,0,0,0
3,"Aragorn, the Uniter",0.660639,3,0,1,0,0,0,0,1,0,0,0,0
4,"Arwen, Mortal Queen",0.002318,4,0,1,0,0,0,0,1,0,0,0,0


In [26]:
process_and_save_in_batches('dataset/train_set.csv', meta_data_card, 'dataset/train',batch_size=42000)
process_and_save_in_batches('dataset/test_set.csv', meta_data_card, 'dataset/test',batch_size=42000)
process_and_save_in_batches('dataset/val_set.csv', meta_data_card, 'dataset/val',batch_size=42000)


In [4]:
import pickle
with open("dataset/test_X_2.pkl", "rb") as f:
            batch_data = pickle.load(f)